## Only 3, medium size lizon's accounts are compared

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Dec  3 02:53:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    46W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import os
import pandas as pd
import numpy as np

In [6]:
########## Ensure reproducibility ##########


# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(42)

#Does not work with ktrain
#os.environ['TF_DETERMINISTIC_OPS'] = '1'

# 2. Set `python` built-in pseudo-random generator at a fixed value
#random.seed(42)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(42)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(42)

In [ ]:
!pip install ktrain

In [8]:
import ktrain
from ktrain import text

In [9]:
Data = '/content/drive/MyDrive/Colab Notebooks/Projects/eva/Data/'

clone_Messages_filename='lizon_clone_data_for_test.csv'
clone_Messages_full_filename=os.path.join(Data, clone_Messages_filename)

lizon_Messages_filename='lizon_data_for_finetuning.csv'
lizon_Messages_full_filename=os.path.join(Data, lizon_Messages_filename)

lizon_ids=[472696, 69715, 300514]

Authors_filename='Authors.csv'
Authors_full_filename= os.path.join(Data, Authors_filename)

## Data load and/or split

In [10]:
clone_df=pd.read_csv(clone_Messages_full_filename, error_bad_lines=False, index_col=False)

In [11]:
lizon_df=pd.read_csv(lizon_Messages_full_filename, error_bad_lines=False, index_col=False)

In [12]:
#to get one author name per Author_Id (they can vary in messages)
Authors=pd.read_csv(Authors_full_filename, error_bad_lines=False, index_col=False)

In [13]:
clone_df = clone_df[clone_df['Author_Id'].isin(lizon_ids)][['Author_Id','original_message']].copy(deep=True)

In [14]:
lizon_df = lizon_df[lizon_df['Author_Id'].isin(lizon_ids)][['Author_Id','original_message']].copy(deep=True)

In [15]:
df = lizon_df.append(clone_df)

In [16]:
df = pd.merge(Authors[['Author_Id','author']], df, left_on='Author_Id', right_on='Author_Id', how='inner')

In [17]:
#split data for train/test
from sklearn import  model_selection

df_trainvalid, df_test = model_selection.train_test_split(df, test_size=0.3, random_state=42,shuffle=True)
df_train, df_valid  = model_selection.train_test_split(df_trainvalid, test_size=0.3, random_state=42,shuffle=True)

In [18]:
df0 = df.groupby(['author']).size().reset_index(name='cnt_total')
df1 = df_train.groupby(['author']).size().reset_index(name='cnt_train')
df2 = df_valid.groupby(['author']).size().reset_index(name='cnt_valid')
df3 = df_test.groupby(['author']).size().reset_index(name='cnt_test')
df4 = pd.merge(df0, df1, left_on='author', right_on='author', how='inner')
df5 = pd.merge(df4, df2, left_on='author', right_on='author', how='inner')
results= pd.merge(df5, df3, left_on='author', right_on='author', how='inner')
results

,author,cnt_total,cnt_train,cnt_valid,cnt_test
0,"Lizon "" **K**",555,279,129,147
1,lizon **,613,302,126,185
2,Кассиапея D,635,302,124,209


In [19]:
x_train = df_train['original_message'].values.astype(str)
x_valid = df_valid['original_message'].values.astype(str)
x_test = df_test['original_message'].values.astype(str)
## get target
y_train = df_train['author'].values.astype(str)
y_valid = df_valid['author'].values.astype(str)
y_test = df_test['author'].values.astype(str)

## Model Training

In [20]:
MODEL_NAME = 'blinoff/roberta-base-russian-v0'
t = text.Transformer(MODEL_NAME, maxlen=256)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [21]:
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_valid, y_valid)
test = t.preprocess_test(x_test, y_test)

preprocessing train...
language: ru
train sequence lengths:
	mean : 69
	95percentile : 156
	99percentile : 289


Downloading:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: ru
test sequence lengths:
	mean : 66
	95percentile : 166
	99percentile : 264


preprocessing test...
language: ru
test sequence lengths:
	mean : 69
	95percentile : 162
	99percentile : 304


In [22]:
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=32)
learner.fit_onecycle(lr=1e-5, epochs=10)

404 Client Error: Not Found for url: https://huggingface.co/blinoff/roberta-base-russian-v0/resolve/main/tf_model.h5
/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:1067: UserWarning: Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. You will need PyTorch installed for this.
  warnings.warn('Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. ' +\


Downloading:   0%|          | 0.00/500M [00:00<?, ?B/s]



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/10
28/28 [==============================] - 27s 325ms/step - loss: 1.0997 - accuracy: 0.3590 - val_loss: 1.0958 - val_accuracy: 0.3272
Epoch 2/10
28/28 [==============================] - 7s 229ms/step - loss: 1.0913 - accuracy: 0.3794 - val_loss: 1.0714 - val_accuracy: 0.5409
Epoch 3/10
28/28 [==============================] - 7s 230ms/step - loss: 1.0599 - accuracy: 0.4553 - val_loss: 1.0321 - val_accuracy: 0.4485
Epoch 4/10
28/28 [==============================] - 7s 229ms/step - loss: 0.9962 - accuracy: 0.5311 - val_loss: 0.9772 - val_accuracy: 0.5646
Epoch 5/10
28/28 [==============================] - 7s 229ms/step - loss: 0.9140 - accuracy: 0.5640 - val_loss: 0.9003 - val_accuracy: 0.5119
Epoch 6/10
28/28 [==============================] - 7s 229ms/step - loss: 0.8079 - accuracy: 0.6308 - val_loss: 0.9010 - val_accuracy: 0.5752
Epoch 7/10
28/28 [==============================] - 7s 229ms/step - loss: 0.7196 - a

## Results

In [23]:
test_confusion_matrix=learner.validate(val_data=test, class_names=t.get_classes())

               precision    recall  f1-score   support

Lizon " **K**       0.35      0.32      0.33       147
     lizon **       0.49      0.56      0.53       185
  Кассиапея D       0.68      0.64      0.66       209

     accuracy                           0.52       541
    macro avg       0.51      0.51      0.51       541
 weighted avg       0.53      0.52      0.52       541



### Confusion Matrix

In [24]:
test_confusion_matrix_df=pd.DataFrame(data=test_confusion_matrix,index=t.get_classes(),columns=t.get_classes())
test_confusion_matrix_df

,"Lizon "" **K**",lizon **,Кассиапея D
"Lizon "" **K**",47,69,31
lizon **,49,104,32
Кассиапея D,38,38,133


### Precision, Recall and F1

In [25]:
!pip install pycm

     |████████████████████████████████| 65 kB 2.5 MB/s eta 0:00:011
     |████████████████████████████████| 574 kB 10.4 MB/s 


In [26]:
import pycm as cm
d_confusion_matrix={}
for i in range(len(t.get_classes())):
  d_confusion_matrix[t.get_classes()[i]]=dict(zip(t.get_classes(), test_confusion_matrix[i]))
d_confusion_matrix=eval(str(d_confusion_matrix))  
model_cm=cm.ConfusionMatrix(matrix=d_confusion_matrix)

In [27]:
results['Precision'] = results['author'].map(model_cm.PPV)
results['Recall'] = results['author'].map(model_cm.TPR)
results['F1'] = results['author'].map(model_cm.F1)
results.sort_values('cnt_total',ascending=False)

,author,cnt_total,cnt_train,cnt_valid,cnt_test,Precision,Recall,F1
2,Кассиапея D,635,302,124,209,0.678571,0.636364,0.656790
1,lizon **,613,302,126,185,0.492891,0.562162,0.525253
0,"Lizon "" **K**",555,279,129,147,0.350746,0.319728,0.334520


### Weighted F1

In [28]:
model_cm.weighted_average('F1')

0.5242425696827829

### Kappa

In [29]:
model_cm.Kappa

0.2795413091240724